In [1]:
import pandas as pd

#### Decripción del Problema

Fraude con tarjetas de crédito: análisis y prevención.

El fraude con tarjetas de crédito representa una amenaza significativa para la integridad de las transacciones financieras y la confianza de los consumidores en el comercio digital. A medida que la dependencia de las tarjetas de crédito para las compras diarias continúa creciendo, también lo hace la sofisticación de los estafadores que explotan las vulnerabilidades del sistema. Este proyecto tiene como objetivo analizar los patrones de fraude con tarjetas de crédito, comprender los factores que contribuyen a las actividades fraudulentas y explorar métodos efectivos para la detección y prevención.

#### Credit Card Fraud Dataset

El **Credit Card Fraud Dataset** es un conjunto de datos que comprende 100.000 transacciones generadas para simular la actividad de las tarjetas de crédito en el mundo real. Cada entrada incluye las siguientes características:

- **TransactionID**:  Un identificador único para cada transacción, que garantiza la trazabilidad.
- **TransactionDate**: la fecha y hora en que se produjo la transacción, lo que permite el análisis temporal.
- **Amount**: el valor monetario de la transacción, que puede ayudar a identificar transacciones inusualmente grandes que pueden indicar fraude.
- **MerchantID**: Un identificador para el comerciante involucrado en la transacción, útil para evaluar patrones de fraude relacionados con el comerciante.
- **TransactionType**: indica si la transacción fue una compra o un reembolso, proporcionando contexto para la actividad.
- **Location**: La ubicación geográfica de la transacción, lo que facilita el análisis de las tendencias de fraude por región.
- **IsFraud**: una variable objetivo binaria que indica si la transacción es fraudulenta (1) o legítima (0), esencial para los modelos de aprendizaje supervisado.

In [7]:
data = pd.read_csv("./data/credit_card_fraud_dataset.csv")
data.head()

,TransactionID,TransactionDate,Amount,MerchantID,TransactionType,Location,IsFraud
0,1,2024-04-03 14:15:35.462794,4189.27,688,refund,San Antonio,0
1,2,2024-03-19 13:20:35.462824,2659.71,109,refund,Dallas,0
2,3,2024-01-08 10:08:35.462834,784.00,394,purchase,New York,0
3,4,2024-04-13 23:50:35.462850,3514.40,944,purchase,Philadelphia,0
4,5,2024-07-12 18:51:35.462858,369.07,475,purchase,Phoenix,0
